This script:
1. Generates all possible combinations of a set of letters (at random locations)
2. Excludes unwanted letter and location patterns
3. Adds columns of correct and incorrect options for each grid


In [18]:
import random
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
import pandas as pd
import itertools
import ast


In [19]:
path_to_save = '...' #! SET YOUR PATH HERE


In [22]:
# Total combinations of letters
i = [i for i in itertools.combinations('ТЛКМДПГЖШФ', 4)]
len(i)

210

In [7]:
# Total combinations of locations
j = [j for j in itertools.combinations(range(8), 4)]
len(j)


70

In [23]:
# Total number of possible stimuli
len(i)*len(j)

14700

In [3]:
# Generate all possible grids (excluding cental position)

def generate_all_grids():
    # Define the set of letters to be used in combinations
    letters = 'ТЛКМДПГЖШФ'

    # List to store all generated grids
    all_grids = []

    # Iterate through all combinations of 4 letters from the set
    for combination in itertools.combinations(letters, 4):
        # Iterate through all combinations of 4 positions on a 3x3 grid
        for positions in itertools.combinations(range(9), 4):
            # Shuffle the current combination
            combination = tuple(random.sample(list(combination), len(list(combination))))
            # Create a 3x3 grid filled with placeholder character 'Х'
            grid = np.full((3, 3), 'Х')

            # If the number 4 (central location) is among the chosen positions, skip this iteration
            if 4 in positions:
                continue  # Skip this iteration

            # Fill the grid with letters at corresponding positions
            for letter, position in zip(combination, positions):
                row, col = divmod(position, 3)
                grid[row, col] = letter

            # Add the generated grid to the list
            all_grids.append(grid)

    return all_grids


In [4]:
# Create pictures from grids

def draw_grid(grid, filename):
    # Flip the grid vertically (to match typical grid orientation)
    flipped_grid = np.flipud(grid)

    # Set figure parameters
    fig, ax = plt.subplots()
    ax.axis('off')
    ax.set_aspect('equal')
    font = {'family': 'Century Gothic'}

    # Draw vertical and horizontal lines to create grid
    for i in range(4):
        ax.vlines(i, 0, 3, color='gray', linewidth=2)  # vertical lines
        ax.hlines(i, 0, 3, color='gray', linewidth=2)  # horizontal lines

    # Fill each cell with light gray color and write the letter
    for i in range(3):
        for j in range(3):
            ax.axvspan(j, j + 1, i, i + 1, facecolor=(0.95, 0.95, 0.95)) # 0.95 or 0.97
            if flipped_grid[i, j] != 'Х':
                rect = plt.Rectangle((j, i), 1, 1, color='0.85')  # light gray rectangle # 0.85 or 0.9
                ax.add_patch(rect)
            ax.text(j + 0.5, i + 0.5, flipped_grid[i, j], fontdict=font, fontsize=30, ha='center', va='center')

    # Save the figure as an image
    plt.savefig(f'{path_to_save}{filename}', bbox_inches='tight', pad_inches=-0.18, dpi=300)
    plt.close()


In [5]:
# Set rules for stimuli generation

def exceptions(letters: list[str, str, str, str], indices: list[int, int, int, int]) -> bool:
    
    # Initialize exception flag
    exception = False
    
    # Define positional indices that lead to exceptions (for symmetrical patterns)
    ind_exc_list = [[0,1,3,4], [1,2,4,5], [3,4,6,7], [4,5,7,8], [0, 2, 6, 8], [1, 3, 5, 7]]
    
    # Check if the provided indices match any of the predefined exceptions
    if indices in ind_exc_list:
        exception = True

    # Define pairs of letters that lead to exceptions
    letter_pairs = [['Г', 'К'], ['Г', 'Т'], ['Л', 'Д'], ['Т', 'Д'], ['Ш', 'Ж'], ['К', 'Л'], ['Л', 'М']]
    
    # Check if any of the letter pairs exist in the provided list of letters
    for pair in letter_pairs:
        if pair[0] in letters and pair[1] in letters:
            # If a pair is found, set exception flag to True and break the loop
            exception = True
            break
    
    # Return the final exception flag
    return exception


In [ ]:
# Generate all grids and save them as CSV (will be used later to generate sequences)

# Generate all grids
all_grids = generate_all_grids()

# Create an empty DataFrame to store grid information
df = pd.DataFrame(columns=['grid', 'letters', 'locs', 'encoding_stim'])

# Loop through each generated grid
for i, grid in enumerate(all_grids):
    # Convert grid to list format and extract non-placeholder letters
    letters = grid.tolist()
    letters = [element for sublist in letters for element in sublist if element != 'Х'] # Remove 'Х' placeholders

    # Get indices of non-placeholder letters
    indices = np.where(grid.flatten() != 'Х')[0].tolist()

    # Check for exceptions based on defined rules
    exception = exceptions(letters, indices)

    # If no exceptions are found, proceed to save the grid
    if exception == False:
        # Define filename for the grid image
        filename = f'grid_{i+1}.png'
        
        # Draw and save the grid image
        draw_grid(grid, filename)

        # Add grid information to the DataFrame
        df = df.append({'grid': [grid], 'letters': letters, 'locs': indices, 'encoding_stim': filename}, ignore_index=True)

    # If exceptions are found, skip to the next grid
    else:
        continue


# Save the DataFrame to a CSV file
df.to_csv(f'{path_to_save}all_grids.csv', sep=';', index=True, encoding='utf-8-sig')


**CHECKING FOR CORRECTNESS**

In [53]:
# Now we would like to add columns with information about correct and incorrect answers to the ALL_GRIDS dataframe
all_grids_path = '...\\all_grids.csv' #! SET YOUR PATH HERE
all_grids_df = pd.read_csv(all_grids_path, delimiter=';', index_col=0)
all_grids_df.head()

,grid,letters,locs,encoding_stim
0,"[array([['Л', 'Ж', 'П'],\n ['Т', 'Х', 'Х...","['Л', 'Ж', 'П', 'Т']","[0, 1, 2, 3]",grid_1331.png
1,"[array([['Ж', 'Л', 'Т'],\n ['Х', 'Х', 'П...","['Ж', 'Л', 'Т', 'П']","[0, 1, 2, 5]",grid_1332.png
2,"[array([['П', 'Л', 'Ж'],\n ['Х', 'Х', 'Х...","['П', 'Л', 'Ж', 'Т']","[0, 1, 2, 6]",grid_1333.png
3,"[array([['Ж', 'Л', 'Т'],\n ['Х', 'Х', 'Х...","['Ж', 'Л', 'Т', 'П']","[0, 1, 2, 7]",grid_1334.png
4,"[array([['Т', 'П', 'Ж'],\n ['Х', 'Х', 'Х...","['Т', 'П', 'Ж', 'Л']","[0, 1, 2, 8]",grid_1335.png


*rules for location*

In [54]:
# Creating a column of incorrect locations

# Define all possible indices (excluding the center)
all_inds = [0, 1, 2, 3, 5, 6, 7, 8]

# Initialize a list to store incorrect locations for each grid
incorrect_locs = []

# Iterate through the 'locs' column of the DataFrame
for i in all_grids_df['locs']:
    sublist = []
    # Convert the string representation of indices to a list
    i = ast.literal_eval(i)
    # Check each index in all_inds
    for j in all_inds:
        # If the index is not present in the 'locs' list, add it to the sublist
        if j not in i:
            sublist.append(j)
    # Append the sublist of incorrect locations to the incorrect_locs list
    incorrect_locs.append(sublist)

# Add the list of incorrect locations as a new column 'incorr_locs' in the DataFrame
all_grids_df['incorr_locs'] = incorrect_locs


*rules for letters*

In [55]:
# Correct and incorrect letters for verbal span

# Initialize lists to store correct and incorrect letters for each grid
corr_letters = []
incorrect_lett = []

# Iterate through the 'letters' column of the DataFrame
for sequence in all_grids_df['letters']:
    # Convert the string representation of the sequence to a list
    sequence = ast.literal_eval(sequence)
    # Convert the list to a string
    sequence = ''.join(sequence)
    
    # Create a dictionary mapping each letter to its index in the sequence
    letter_dict = {letter: index for index, letter in enumerate(sequence)}
    
    # Create a dictionary mapping each letter to a list of indices where it appears incorrectly
    incorrect_indices_dict = {
        letter: [index for index, char in enumerate(sequence) if char != letter] for letter in sequence
    }
    
    # Append the dictionaries to the respective lists
    corr_letters.append(letter_dict)
    incorrect_lett.append(incorrect_indices_dict)

# Add the lists of correct and incorrect letters as new columns in the DataFrame
all_grids_df['corr_letters'] = corr_letters
all_grids_df['incorr_letters'] = incorrect_lett


*rules for rotation*

In [56]:
def rotate_indices_clockwise(indices: list[int]) -> list[int]:
    # Define a dictionary that maps each index to its clockwise rotated position
    transformation_dict = {0: 2, 1: 5, 2: 8, 3: 1, 5: 7, 6: 0, 7: 3, 8: 6}
    
    # Use list comprehension to generate the rotated indices
    rotated_indices = [transformation_dict[index] for index in indices]
    
    # Return the rotated indices
    return rotated_indices


In [57]:
# Correct and incorrect rotation

# Define all possible indices (excluding the center)
all_inds = [0, 1, 2, 3, 5, 6, 7, 8]

# Initialize lists to store rotated indices and incorrect rotations for each grid
rotated_inds = []
incorrect_rot = []

# Iterate through the 'locs' column of the DataFrame
for locs in all_grids_df['locs']:
    sublist = []
    # Convert the string representation of indices to a list
    locs = ast.literal_eval(locs)
    # Rotate the indices clockwise
    rotation = rotate_indices_clockwise(locs)
    # Append the rotated indices to the list
    rotated_inds.append(rotation)

    # Check each index in all_inds
    for j in all_inds:
        # If the index is not present in the rotated indices, add it to the sublist
        if j not in rotation:
            sublist.append(j)
    # Append the sublist of incorrect rotations to the incorrect_rot list
    incorrect_rot.append(sublist)

# Add the list of rotated indices as a new column 'rotated_inds' in the DataFrame
all_grids_df['rotated_inds'] = rotated_inds

# Add the list of incorrect rotations as a new column 'incorr_rot' in the DataFrame
all_grids_df['incorr_rot'] = incorrect_rot

In [12]:
all_grids_df.head()

,grid,letters,locs,encoding_stim,incorr_locs,corr_letters,incorr_letters,rotated_inds,incorr_rot
0,"[array([['Л', 'Ж', 'П'],\n ['Т', 'Х', 'Х...","['Л', 'Ж', 'П', 'Т']","[0, 1, 2, 3]",grid_1331.png,"[5, 6, 7, 8]","{'Л': 0, 'Ж': 1, 'П': 2, 'Т': 3}","{'Л': [1, 2, 3], 'Ж': [0, 2, 3], 'П': [0, 1, 3...","[2, 5, 8, 1]","[0, 3, 6, 7]"
1,"[array([['Ж', 'Л', 'Т'],\n ['Х', 'Х', 'П...","['Ж', 'Л', 'Т', 'П']","[0, 1, 2, 5]",grid_1332.png,"[3, 6, 7, 8]","{'Ж': 0, 'Л': 1, 'Т': 2, 'П': 3}","{'Ж': [1, 2, 3], 'Л': [0, 2, 3], 'Т': [0, 1, 3...","[2, 5, 8, 7]","[0, 1, 3, 6]"
2,"[array([['П', 'Л', 'Ж'],\n ['Х', 'Х', 'Х...","['П', 'Л', 'Ж', 'Т']","[0, 1, 2, 6]",grid_1333.png,"[3, 5, 7, 8]","{'П': 0, 'Л': 1, 'Ж': 2, 'Т': 3}","{'П': [1, 2, 3], 'Л': [0, 2, 3], 'Ж': [0, 1, 3...","[2, 5, 8, 0]","[1, 3, 6, 7]"
3,"[array([['Ж', 'Л', 'Т'],\n ['Х', 'Х', 'Х...","['Ж', 'Л', 'Т', 'П']","[0, 1, 2, 7]",grid_1334.png,"[3, 5, 6, 8]","{'Ж': 0, 'Л': 1, 'Т': 2, 'П': 3}","{'Ж': [1, 2, 3], 'Л': [0, 2, 3], 'Т': [0, 1, 3...","[2, 5, 8, 3]","[0, 1, 6, 7]"
4,"[array([['Т', 'П', 'Ж'],\n ['Х', 'Х', 'Х...","['Т', 'П', 'Ж', 'Л']","[0, 1, 2, 8]",grid_1335.png,"[3, 5, 6, 7]","{'Т': 0, 'П': 1, 'Ж': 2, 'Л': 3}","{'Т': [1, 2, 3], 'П': [0, 2, 3], 'Ж': [0, 1, 3...","[2, 5, 8, 6]","[0, 1, 3, 7]"


*rules for alphabet*

In [58]:
# Correct alphabetical order

# Initialize lists to store alphabetically ordered letters and incorrect alphabetical orders for each grid
abc_ordered = []
incorrect_abc = []

# Iterate through the 'letters' column of the DataFrame
for sequence in all_grids_df['letters']:
    # Convert the string representation of the sequence to a list
    sequence = ast.literal_eval(sequence)
    # Sort the sequence alphabetically
    sequence = ''.join(sorted(sequence))
    
    # Create a dictionary mapping each letter to its index in the alphabetically ordered sequence
    letter_dict = {letter: index for index, letter in enumerate(sequence)}
    
    # Create a dictionary mapping each letter to a list of indices where it appears incorrectly
    incorrect_indices_dict = {
        letter: [index for index, char in enumerate(sequence) if char != letter] for letter in sequence
    }
    
    # Append the dictionaries to the respective lists
    abc_ordered.append(letter_dict)
    incorrect_abc.append(incorrect_indices_dict)

# Add the lists of alphabetically ordered letters as a new column 'abc_ordered' in the DataFrame
all_grids_df['abc_ordered'] = abc_ordered

*additional rule for the incorrect ABC locations*

In [59]:
# We would like to limit the positions where a letter can be incorrectly presented

# 1. Arrange the set of letters alphabetically and split in 4 groups
# Define the set of letters alphabetically
all_letters = 'ТЛКМДПГЖШФ'
# Sort the letters alphabetically
all_letters_sorted = sorted(all_letters)
# Define the sizes of each group
group_sizes = [2, 3, 3, 2]
# Initialize a list to store groups of letters
groups = []

# Split the sorted letters into groups based on the group sizes
for i in range(len(group_sizes)):
    groups.append(tuple(all_letters_sorted[sum(group_sizes[:i]):sum(group_sizes[:i+1])]))

# 2. Decide possible location for each group
possible_locs = [[0, 1], [0, 1, 2], [1, 2, 3], [2, 3]]
# Create a dictionary mapping each group of letters to its possible locations
abc_groups_dict = {key: value for key, value in zip(groups, possible_locs)}

# 3. Create a dictionary with possible locations for each letter
incorr_abc_rule = {}
for letters, positions in abc_groups_dict.items():
    for letter in letters:
        # Map each letter to its possible locations
        incorr_abc_rule[letter] = positions


In [65]:
incorr_abc_rule

{'Г': [0, 1],
 'Д': [0, 1],
 'Ж': [0, 1, 2],
 'К': [0, 1, 2],
 'Л': [0, 1, 2],
 'М': [1, 2, 3],
 'П': [1, 2, 3],
 'Т': [1, 2, 3],
 'Ф': [2, 3],
 'Ш': [2, 3]}

In [60]:
# Filter incorrect positions for letters based on the rule in the previous cells
def filtered_incorr_dict(incorrect_abc: list[dict], incorr_abc_rule: dict) -> dict:
    # Create a new dictionary to store the filtered incorrect locations
    filtered_dicts = []

    for abc_dict in incorrect_abc:
        filtered_incorr_dict = {}
        # Iterate through the keys of incorr_dict
        for letter, positions in abc_dict.items():
            # Get the allowed positions from incorr_abc_rule for the current letter
            allowed_positions = incorr_abc_rule.get(letter, [])

            # Find the intersection between allowed positions and the incorrect positions
            filtered_positions = list(set(positions) & set(allowed_positions))

            # Update the filtered_incorr_dict with the filtered positions
            if filtered_positions:
                filtered_incorr_dict[letter] = filtered_positions

        filtered_dicts.append(filtered_incorr_dict)

    return filtered_dicts

In [61]:
# Incorrect alphabetical order

filtered_incorr_dict = filtered_incorr_dict(incorrect_abc, incorr_abc_rule)

# Assign the filtered dictionaries to a new column 'incorr_abc' in the DataFrame
all_grids_df['incorr_abc'] = filtered_incorr_dict

In [62]:
all_grids_df.head()

,grid,letters,locs,encoding_stim,incorr_locs,corr_letters,incorr_letters,rotated_inds,incorr_rot,abc_ordered,incorr_abc
0,"[array([['Л', 'Ж', 'П'],\n ['Т', 'Х', 'Х...","['Л', 'Ж', 'П', 'Т']","[0, 1, 2, 3]",grid_1331.png,"[5, 6, 7, 8]","{'Л': 0, 'Ж': 1, 'П': 2, 'Т': 3}","{'Л': [1, 2, 3], 'Ж': [0, 2, 3], 'П': [0, 1, 3...","[2, 5, 8, 1]","[0, 3, 6, 7]","{'Ж': 0, 'Л': 1, 'П': 2, 'Т': 3}","{'Ж': [1, 2], 'Л': [0, 2], 'П': [1, 3], 'Т': [..."
1,"[array([['Ж', 'Л', 'Т'],\n ['Х', 'Х', 'П...","['Ж', 'Л', 'Т', 'П']","[0, 1, 2, 5]",grid_1332.png,"[3, 6, 7, 8]","{'Ж': 0, 'Л': 1, 'Т': 2, 'П': 3}","{'Ж': [1, 2, 3], 'Л': [0, 2, 3], 'Т': [0, 1, 3...","[2, 5, 8, 7]","[0, 1, 3, 6]","{'Ж': 0, 'Л': 1, 'П': 2, 'Т': 3}","{'Ж': [1, 2], 'Л': [0, 2], 'П': [1, 3], 'Т': [..."
2,"[array([['П', 'Л', 'Ж'],\n ['Х', 'Х', 'Х...","['П', 'Л', 'Ж', 'Т']","[0, 1, 2, 6]",grid_1333.png,"[3, 5, 7, 8]","{'П': 0, 'Л': 1, 'Ж': 2, 'Т': 3}","{'П': [1, 2, 3], 'Л': [0, 2, 3], 'Ж': [0, 1, 3...","[2, 5, 8, 0]","[1, 3, 6, 7]","{'Ж': 0, 'Л': 1, 'П': 2, 'Т': 3}","{'Ж': [1, 2], 'Л': [0, 2], 'П': [1, 3], 'Т': [..."
3,"[array([['Ж', 'Л', 'Т'],\n ['Х', 'Х', 'Х...","['Ж', 'Л', 'Т', 'П']","[0, 1, 2, 7]",grid_1334.png,"[3, 5, 6, 8]","{'Ж': 0, 'Л': 1, 'Т': 2, 'П': 3}","{'Ж': [1, 2, 3], 'Л': [0, 2, 3], 'Т': [0, 1, 3...","[2, 5, 8, 3]","[0, 1, 6, 7]","{'Ж': 0, 'Л': 1, 'П': 2, 'Т': 3}","{'Ж': [1, 2], 'Л': [0, 2], 'П': [1, 3], 'Т': [..."
4,"[array([['Т', 'П', 'Ж'],\n ['Х', 'Х', 'Х...","['Т', 'П', 'Ж', 'Л']","[0, 1, 2, 8]",grid_1335.png,"[3, 5, 6, 7]","{'Т': 0, 'П': 1, 'Ж': 2, 'Л': 3}","{'Т': [1, 2, 3], 'П': [0, 2, 3], 'Ж': [0, 1, 3...","[2, 5, 8, 6]","[0, 1, 3, 7]","{'Ж': 0, 'Л': 1, 'П': 2, 'Т': 3}","{'Ж': [1, 2], 'Л': [0, 2], 'П': [1, 3], 'Т': [..."


In [63]:
# Save the resulting DataFrame as a CSV file
all_grids_df.to_csv(all_grids_path, sep=';', index=True, encoding='utf-8-sig')

*drawing single squares for PROBES*

In [7]:
# We would like to create visual probes that will be presented to the participants
probes_dir = '...\\probes\\' #! SET YOUR PATH HERE

In [ ]:
# Draw PROBES with single locations highlighted

# Dictionary mapping indices to grid positions
loc_index = {
            0: (0, 2),
            1: (1, 2),
            2: (2, 2),
            3: (0, 1),
            4: (1, 1),
            5: (2, 1),
            6: (0, 0),
            7: (1, 0),
            8: (2, 0)
            }

# Create an empty DataFrame to store metadata
data = pd.DataFrame({'shaded_index': [], 'filename': []})

# Loop through each key-value pair in loc_index
for key, value in loc_index.items():
    fig, ax = plt.subplots()
    ax.axis('off')
    ax.set_aspect('equal')

    # Draw the grid lines
    for i in range(4):
        ax.hlines(i, 0, 3, color='gray', linewidth=2)  # horizontal lines
        ax.vlines(i, 0, 3, color='gray', linewidth=2)  # vertical lines

     # Determine the position to shade based on the current key
    position = value
    ax.axvspan(0, 3, facecolor=(0.95, 0.95, 0.95)) # 0.95 or 0.97
    rect = plt.Rectangle(position, 1, 1, color='0.85') # light gray rectangle # 0.85 or 0.9
    ax.add_patch(rect)

    # Save the plot as an image
    if key != 4:
        plt.savefig(f'{probes_dir}figure_{key}.png', bbox_inches='tight', pad_inches=-0.18, dpi=300)
        plt.close()

        data = data.convert_dtypes().append({'shaded_index': int(key), 'filename': f'figure_{key}.png'}, ignore_index=True)

# Save metadata as a CSV file
data.to_csv(f'{probes_dir}fig_df.csv', sep=';', index=True, encoding='utf-8-sig')


**THE END**